In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    model="gpt-4o-mini",   # usa un modelo que tengas habilitado
    temperature=0
)
print("🤖 Modelo listo")

🤖 Modelo listo


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

In [4]:
import os, json
from datetime import datetime

# === Función para auditar código ===
def audit_code(file_path: str):
    """
    Realiza una auditoría de seguridad del archivo indicado
    usando el modelo inicializado en LangChain.
    """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        code = f.read()

    prompt_template = """Eres un experto en seguridad OWASP.
    Analiza el siguiente código en busca de vulnerabilidades y responde en JSON válido.
    El formato debe ser:
    {{
      "file": "{file_name}",
      "vulnerabilities": [
        {{
          "title": "...",
          "severity": "CRITICAL|HIGH|MEDIUM|LOW",
          "cwe_id": "CWE-###",
          "description": "...",
          "line_numbers": [<int>],
          "recommendation": "..."
        }}
      ]
    }}

    Código:
    ```{code_lang}
    {code}
    ```"""

    # Crea el prompt
    prompt = prompt_template.format(
        file_name=os.path.basename(file_path),
        code_lang="C# (ASP.NET WebForms)",
        code=code[:4000]  # truncar si el archivo es muy largo
    )

    response = model.invoke(prompt)

    # Guarda el resultado en carpeta results
    os.makedirs("results", exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_file = f"results/{os.path.basename(file_path)}_audit_{ts}.json"

    try:
        data = json.loads(response.content)
    except Exception:
        data = {"error": "Respuesta no es JSON válido", "raw": response.content}

    with open(out_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Auditoría guardada en {out_file}")
    return data


In [5]:
import glob

# Buscar archivos .cs, .aspx, .asax dentro de /codigos
files = []
for ext in ("*.cs", "*.aspx", "*.asax", "*.config"):
    files.extend(glob.glob(os.path.join("codigos", "**", ext), recursive=True))

print(f"Encontrados {len(files)} archivos para auditar")

# Auditar todos
for f in files:
    print(f"🔍 Auditando: {f}")
    audit_code(f)


Encontrados 0 archivos para auditar


In [6]:
import json, glob

summary = {"CRITICAL":0, "HIGH":0, "MEDIUM":0, "LOW":0, "UNKNOWN":0}

for file in glob.glob("results/*_audit_*.json"):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for v in data.get("vulnerabilities", []):
            sev = v.get("severity", "UNKNOWN").upper()
            if sev in summary: summary[sev]+=1
            else: summary["UNKNOWN"]+=1

print("📊 Resumen general:")
for s, n in summary.items():
    print(f"{s:9}: {n}")


📊 Resumen general:
CRITICAL : 0
HIGH     : 0
MEDIUM   : 0
LOW      : 0
UNKNOWN  : 0
